In [2]:
import cv2
from ultralytics import YOLO
import random
import pandas as pd


# Initialize the YOLO model
model = YOLO("/Users/jatavathpavannaik/Documents/python/Computer_Vision/YOLOV8/runs/detect/train_cpu/weights/best.pt")

# Open video capture
cap = cv2.VideoCapture('/Users/jatavathpavannaik/Documents/python/Computer_Vision/test_case_03.mov')  # Replace 'path_to_video.mp4' with your video file
  # Replace 'path_to_video.mp4' with your video file
fps = cap.get(cv2.CAP_PROP_FPS)
# Create a list to store areas of bounding boxes for each frame
all_areas = []

frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_number += 1
    
    # Make predictions
    results = model.predict(frame)
    
    # Extract bounding box information
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()
    
    # Draw bounding boxes on the frame and calculate areas
    areas=[]
    for box, cls, conf in zip(boxes, classes, confidences):
        x1, y1, x2, y2 = box
        class_name = names[int(cls)]
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, f'{class_name} {conf:.2f}', (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Calculate area
        area = (x2 - x1) * (y2 - y1)
        areas.append(area)
    
    # Store areas of bounding boxes for this frame
    all_areas.append((frame_number, areas))
    
    # Display the frame with bounding boxes
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 51.1ms
Speed: 5.2ms preprocess, 51.1ms inference, 10.3ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 73.1ms
Speed: 1.0ms preprocess, 73.1ms inference, 0.6ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 26.0ms
Speed: 1.0ms preprocess, 26.0ms inference, 0.4ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 33.4ms
Speed: 0.7ms preprocess, 33.4ms inference, 0.4ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 27.7ms
Speed: 0.8ms preprocess, 27.7ms inference, 0.5ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 28.3ms
Speed: 0.6ms preprocess, 28.3ms inference, 0.4ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 1 CTA, 1 ad, 1 badge, 1 card, 1 pop-up, 29.0ms
Spee

In [3]:

# Assuming all_areas is a list of tuples where each tuple contains (frame_name, list_of_areas)
sum_data = [(tup[0], sum(tup[1]), len(tup[1]), tup[1]) for tup in all_areas]

# Create DataFrame
Frame_Data_df = pd.DataFrame(sum_data, columns=['Frame', 'Total_Area', 'No_of_Bounding_Boxes', 'Individual_Areas'])

In [4]:
Frame_Data_df

,Frame,Total_Area,No_of_Bounding_Boxes,Individual_Areas
0,1,769530.974652,5,"[419207.82495868206, 20738.51675581187, 116922..."
1,2,782786.455085,5,"[419308.08666467667, 20764.720623265952, 11728..."
2,3,782881.769880,5,"[419217.05669403076, 20769.6156077981, 117419...."
3,4,769508.750484,5,"[419190.43410375714, 20768.261165644974, 11744..."
4,5,769386.451053,5,"[20770.498541947454, 419099.34338368475, 11746..."
...,...,...,...,...
2601,2602,314540.224304,3,"[22140.36712399125, 20439.48525609076, 271960...."
2602,2603,314636.428282,3,"[22139.785792909563, 20442.683440413326, 27205..."
2603,2604,314615.686923,3,"[22143.308959431946, 20440.200539585203, 27203..."
2604,2605,314679.112757,3,"[22139.3238438908, 20435.44445732981, 272104.3..."


In [5]:
def find_duplicate_frames(df):
        start_index = None
        end_index = None
        duplicate_series = []

        for index, row in df.iterrows():
            if index > 0:
                prev_row = df.iloc[index - 1]
                if row['No_of_Bounding_Boxes'] == prev_row['No_of_Bounding_Boxes']:
                    areas_current = row['Individual_Areas']
                    areas_prev = prev_row['Individual_Areas']
                    all_diffs_less_than_100 = all(abs(area_curr - area_prev) < 100 for area_curr, area_prev in zip(areas_current, areas_prev))
                    if all_diffs_less_than_100:
                        if start_index is None:
                            start_index = index - 1
                        end_index = index
                else:
                    if start_index is not None:
                        duplicate_series.append((start_index, end_index))
                        start_index = None
                        end_index = None

        if start_index is not None:
            duplicate_series.append((start_index, end_index))

        if duplicate_series:
            return duplicate_series
        else:
            print("No Duplicated Frames Found")
duplicate_series = find_duplicate_frames(Frame_Data_df)

In [11]:
Frame_Data_df.shape

(2606, 4)

In [6]:
duplicate_series

[(4, 6),
 (7, 8),
 (16, 21),
 (24, 29),
 (42, 43),
 (75, 76),
 (218, 220),
 (245, 247),
 (272, 276),
 (329, 330),
 (358, 361),
 (373, 374),
 (392, 393),
 (443, 444),
 (460, 497),
 (502, 573),
 (574, 575),
 (585, 589),
 (590, 591),
 (595, 596),
 (606, 706),
 (711, 716),
 (718, 734),
 (738, 750),
 (752, 770),
 (782, 868),
 (872, 879),
 (882, 883),
 (886, 887),
 (982, 983),
 (1019, 1359),
 (1360, 1368),
 (1374, 1382),
 (1385, 1398),
 (1401, 1518),
 (1521, 1522),
 (1525, 1534),
 (1536, 1574),
 (1592, 1613),
 (1615, 1621),
 (1624, 1629),
 (1630, 1644),
 (1648, 1649),
 (1656, 1681),
 (1699, 1710),
 (1712, 1721),
 (1726, 1814),
 (1815, 2110),
 (2114, 2122),
 (2128, 2242),
 (2244, 2250),
 (2253, 2258),
 (2262, 2282),
 (2285, 2286),
 (2288, 2328),
 (2331, 2332),
 (2333, 2442),
 (2567, 2592),
 (2594, 2604)]

In [7]:
def delete_frames_in_duplicate_series(Frame_Data_df, duplicate_series, frame_column='Frame'):
    df_cleaned = Frame_Data_df.copy()
    for start, end in duplicate_series:
        frames_to_delete = list(range(start, end + 1))
        df_cleaned = df_cleaned[~df_cleaned[frame_column].isin(frames_to_delete)]
    return df_cleaned
df_cleaned=delete_frames_in_duplicate_series(Frame_Data_df,duplicate_series)

In [8]:
def select_frames_with_threshold(duplicate_series, Frame_Data_df, fps, threshold_seconds):
    frames = []
    threshold_frames = int(threshold_seconds * fps)  # Convert threshold to frames

    for start, end in duplicate_series:
        if end - start > threshold_frames:
            random_index = random.randint(start, end)
            random_row = Frame_Data_df.iloc[random_index]
            frames.append(random_row)

    if frames:
        req_frames = pd.DataFrame(frames)
        return req_frames
    else:
        return None
req_frames=select_frames_with_threshold(duplicate_series,Frame_Data_df,fps,threshold_seconds=0.1)

In [9]:
Frames_to_Retrieve_df=pd.concat([df_cleaned,req_frames])
Frames_to_Retrieve=Frames_to_Retrieve_df["Frame"].astype(int).to_list()


In [10]:
Frames_to_Retrieve_df

,Frame,Total_Area,No_of_Bounding_Boxes,Individual_Areas
0,1,769530.974652,5,"[419207.82495868206, 20738.51675581187, 116922..."
1,2,782786.455085,5,"[419308.08666467667, 20764.720623265952, 11728..."
2,3,782881.769880,5,"[419217.05669403076, 20769.6156077981, 117419...."
8,9,565633.950962,4,"[20796.752253759652, 419369.8316166401, 117723..."
9,10,764029.165535,4,"[20783.654214393348, 419406.1090834141, 117482..."
...,...,...,...,...
2270,2271,267432.903361,3,"[22101.744570415467, 20328.540172219276, 22500..."
2321,2322,268751.886554,3,"[22083.987729446962, 20313.754096627235, 22635..."
2438,2439,268055.356413,3,"[22125.254076430574, 20364.533688593656, 22556..."
2581,2582,314657.767139,3,"[22146.444811904803, 20433.412101387978, 27207..."


In [14]:
import cv2
import os

def retrieve_frames(video_path, frames_to_retrieve, output_video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Check if the video file was successfully opened
    if not video.isOpened():
        print("Error: Unable to open video file")
        return
    
    # Get frames per second
    fps = video.get(cv2.CAP_PROP_FPS)
    
    
    # List to store retrieved frames
    retrieved_frames = []
    
    # Iterate over each frame in the video
    frame_count = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Check if the current frame number is in the list of frames to retrieve
        if frame_count in frames_to_retrieve:
            retrieved_frames.append(frame)
            
        # Display progress
        print(f"Processing frame {frame_count} / {int(video.get(cv2.CAP_PROP_FRAME_COUNT))}", end="\r")
    
    # Release the video object
    video.release()
    
    # Write the retrieved frames to a new video file
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (retrieved_frames[0].shape[1], retrieved_frames[0].shape[0]))
    for frame in retrieved_frames:
        out_video.write(frame)
    out_video.release()
    print("\nVideo created successfully!")

video_path = "/Users/jatavathpavannaik/Documents/python/Computer_Vision/test_case_03.mov"
output_video_path = "output_video.mp4"

retrieve_frames(video_path, Frames_to_Retrieve, output_video_path)


Processing frame 2606 / 2609
Video created successfully!


: 

# Automated

In [ ]:
import cv2
from ultralytics import YOLO
import random
import pandas as pd

def process_video(video_path, output_video_path, threshold_seconds=1):
    # Open video capture
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create a VideoWriter object to write the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Create DataFrame to store frame data
    all_areas = []
    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_number += 1

        # Make predictions
        results = model.predict(frame)

        # Extract bounding box information
        boxes = results[0].boxes.xyxy.tolist()
        classes = results[0].boxes.cls.tolist()
        names = results[0].names
        confidences = results[0].boxes.conf.tolist()

        # Draw bounding boxes on the frame and calculate areas
        areas = []
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            class_name = names[int(cls)]
            color = (0, 255, 0)  # Green color for bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(frame, f'{class_name} {conf:.2f}', (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Calculate area
            area = (x2 - x1) * (y2 - y1)
            areas.append(area)

        # Store areas of bounding boxes for this frame
        all_areas.append((frame_number, areas))

        # Write the frame to output video
        out_video.write(frame)

    # Release resources
    cap.release()
    out_video.release()
    cv2.destroyAllWindows()

    # Create DataFrame with frame data
    sum_data = [(tup[0], sum(tup[1]), len(tup[1]), tup[1]) for tup in all_areas]
    Frame_Data_df = pd.DataFrame(sum_data, columns=['Frame', 'Total_Area', 'No_of_Bounding_Boxes', 'Individual_Areas'])

    # Find duplicate frames
    duplicate_series = find_duplicate_frames(Frame_Data_df)

    # Delete frames in duplicate series
    df_cleaned = delete_frames_in_duplicate_series(Frame_Data_df, duplicate_series)

    # Select frames with threshold
    req_frames = select_frames_with_threshold(duplicate_series, Frame_Data_df, fps, threshold_seconds)

    # Concatenate cleaned DataFrame and required frames DataFrame
    Frames_to_Retrieve_df = pd.concat([df_cleaned, req_frames])

    # Retrieve frames
    Frames_to_Retrieve = Frames_to_Retrieve_df["Frame"].astype(int).to_list()
    retrieve_frames(video_path, Frames_to_Retrieve, output_video_path)

def find_duplicate_frames(df):
    start_index = None
    end_index = None
    duplicate_series = []

    for index, row in df.iterrows():
        if index > 0:
            prev_row = df.iloc[index - 1]
            if row['No_of_Bounding_Boxes'] == prev_row['No_of_Bounding_Boxes']:
                areas_current = row['Individual_Areas']
                areas_prev = prev_row['Individual_Areas']
                all_diffs_less_than_100 = all(abs(area_curr - area_prev) < 100 for area_curr, area_prev in zip(areas_current, areas_prev))
                if all_diffs_less_than_100:
                    if start_index is None:
                        start_index = index - 1
                    end_index = index
            else:
                if start_index is not None:
                    duplicate_series.append((start_index, end_index))
                    start_index = None
                    end_index = None

    if start_index is not None:
        duplicate_series.append((start_index, end_index))

    if duplicate_series:
        return duplicate_series
    else:
        print("No Duplicated Frames Found")

def delete_frames_in_duplicate_series(Frame_Data_df, duplicate_series, frame_column='Frame'):
    df_cleaned = Frame_Data_df.copy()
    for start, end in duplicate_series:
        frames_to_delete = list(range(start, end + 1))
        df_cleaned = df_cleaned[~df_cleaned[frame_column].isin(frames_to_delete)]
    return df_cleaned

def select_frames_with_threshold(duplicate_series, Frame_Data_df, fps, threshold_seconds):
    frames = []
    threshold_frames = int(threshold_seconds * fps)  # Convert threshold to frames

    for start, end in duplicate_series:
        if end - start > threshold_frames:
            random_index = random.randint(start, end)
            random_row = Frame_Data_df.iloc[random_index]
            frames.append(random_row)

    if frames:
        req_frames = pd.DataFrame(frames)
        return req_frames
    else:
        return None
    
model = YOLO("/Users/jatavathpavannaik/Documents/python/Computer_Vision/YOLOV8/runs/detect/train_cpu/weights/best.pt")
process_video('/Users/jatavathpavannaik/Documents/python/Computer_Vision/test_case_2.mov', 'output_video.mp4')
